In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "2호모델"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Q"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'2호모델_Q_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
list_data["2012-01"].columns

Index(['BW금액', 'CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '유상증자금액', '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액',
       '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
divide_data = tools.dict_data_divide(list_data.copy(), divide_what=['BW금액', 'CB금액','유상증자금액',
'최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', 
'타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액',
'당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름','판매 및 관리비'], divide_into="총자본", return_origianl=True)       
divide_data["2021-01"].columns

Index(['BW금액/총자본', 'CB금액/총자본', '유상증자금액/총자본', '최대주주변경을수반하는주식담보제공금액/총자본',
       '최대주주변경을수반하는주식양수도금액/총자본', '타법인주식및출자증권취득결정금액/총자본', '특수관계인에대한자금대여금액/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본', 'BW금액', 'CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트',
       '업종변경카운트', '유상증자금액', '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액',
       '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트', '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
divide_data1 = tools.dict_data_drop(divide_data.copy(), drop_col=['BW금액', 'CB금액','유상증자금액',
'최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액',
'타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액',
'당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름','판매 및 관리비'], dropna=False)

divide_data1["2012-01"].columns

Index(['BW금액/총자본', 'CB금액/총자본', '유상증자금액/총자본', '최대주주변경을수반하는주식담보제공금액/총자본',
       '최대주주변경을수반하는주식양수도금액/총자본', '타법인주식및출자증권취득결정금액/총자본', '특수관계인에대한자금대여금액/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '차입금의존도', '총자본',
       '최대주주비율', '최대주주비율_변동성'],
      dtype='object')

In [12]:
drop_togehter = tools.dict_data_drop(divide_data1.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)
scaled_together["2012-01"].columns

Index(['BW금액/총자본', 'CB금액/총자본', '유상증자금액/총자본', '최대주주변경을수반하는주식담보제공금액/총자본',
       '최대주주변경을수반하는주식양수도금액/총자본', '타법인주식및출자증권취득결정금액/총자본', '특수관계인에대한자금대여금액/총자본',
       '당기순이익/총자본', '매출채권/총자본', '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본',
       '판매 및 관리비/총자본', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '차입금의존도', '총자본',
       '최대주주비율', '최대주주비율_변동성'],
      dtype='object')

In [13]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:07<00:00, 12.67it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 85,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=85)},
 '2012-04': {'num_of_cluseer': 3,
  'random_state': 31,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=31)},
 '2012-07': {'num_of_cluseer': 3,
  'random_state': 44,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=44)},
 '2012-10': {'num_of_cluseer': 3,
  'random_state': 30,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=30)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 73,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=73)},
 '2013-04': {'num_of_cluseer': 3,
  'random_state': 1,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=1)},
 '2013-07': {'num_of_cluseer': 3,
  'random_state': 36,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=36)},
 '2013-10': {'num_of_cluseer': 3,
  'random_state': 89,
  'model': KMeans(max_iter=50

#### 모델 기록

In [14]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

845.0
429.0


,2012-01,2012-04,2012-07,2012-10,2013-01,2013-04,2013-07,2013-10,2014-01,2014-04,...,2020-04,2020-07,2020-10,2021-01,2021-04,2021-07,2021-10,2022-01,2022-04,2022-07
0,482.0,557.0,501.0,497.0,599.0,440.0,552.0,588.0,430.0,464.0,...,749.0,665.0,764.0,647.0,588.0,676.0,671.0,675.0,791.0,681.0
1,535.0,611.0,650.0,584.0,522.0,524.0,429.0,540.0,635.0,585.0,...,551.0,578.0,634.0,589.0,794.0,770.0,789.0,606.0,641.0,513.0
2,577.0,436.0,456.0,527.0,484.0,641.0,619.0,488.0,553.0,569.0,...,676.0,756.0,616.0,789.0,661.0,610.0,595.0,775.0,615.0,845.0
기업수,1594.0,1604.0,1607.0,1608.0,1605.0,1605.0,1600.0,1616.0,1618.0,1618.0,...,1976.0,1999.0,2014.0,2025.0,2043.0,2056.0,2055.0,2056.0,2047.0,2039.0


In [15]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [16]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [17]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [18]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.8036415335017496
1.0


0.10031016393958342

In [19]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

영업이익/총자본                   0.189472
당기순이익/총자본                  0.182974
매출총이익/총자본                  0.122391
총자본                        0.095787
판매 및 관리비/총자본               0.092885
영업현금흐름/총자본                 0.090549
최대주주비율                     0.068203
차입금의존도                     0.044990
매출채권/총자본                   0.040833
CB금액/총자본                   0.021693
결산기간_주가변동률                 0.016632
최대주주비율_변동성                 0.009456
조회공시요구카운트                  0.006282
최대주주변경카운트                  0.004441
결산기간_거래량증가                 0.002903
불성실공시법인지정카운트               0.002126
타법인주식및출자증권취득결정금액/총자본       0.002076
BW금액/총자본                   0.001773
유상증자금액/총자본                 0.001718
최대주주변경을수반하는주식양수도금액/총자본     0.000985
상호변경카운트                    0.000686
최대주주변경을수반하는주식담보제공금액/총자본    0.000646
경영권분쟁소송                    0.000340
업종변경카운트                    0.000133
특수관계인에대한자금대여금액/총자본         0.000024
dtype: float64

예측하는 거 만들기

사용할 기간을 주면 -> 이후 가능한 끝까지의 값을 예측하고, 결과를 리턴하기

In [20]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum
c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2012-04예측,2012-07예측,2012-10예측,2013-01예측,2013-04예측,2013-07예측,2013-10예측,2014-01예측,2014-04예측,2014-07예측,...,2019-07예측,2019-10예측,2020-01예측,2020-04예측,2020-07예측,2020-10예측,2021-01예측,2021-04예측,2021-07예측,2021-10예측
0,0.142857,0.1,0.027778,0.085714,0.090909,0.973684,0.690476,0.023256,0.212766,0.837209,...,0.111111,0.823529,0.080460,0.035294,0.038462,0.126761,0.123077,0.128205,0.800000,0.0
1,0.071429,0.1,0.805556,0.085714,0.000000,0.026316,0.285714,0.046512,0.021277,0.069767,...,0.818182,0.105882,0.091954,0.129412,0.846154,0.788732,0.138462,0.717949,0.133333,0.5
2,0.785714,0.8,0.166667,0.828571,0.909091,0.000000,0.023810,0.930233,0.765957,0.093023,...,0.070707,0.070588,0.827586,0.835294,0.115385,0.084507,0.738462,0.153846,0.066667,0.5


고위험 클러스터의 평균 재무, 공시 상황 출력

* eval_cluster Result의 결과를 받는다 / model_save_dict / raw_data -> drop_together (스케일 전 데이터를 받아야함)

In [21]:
dict = evaluate.see_cluster_info(model_save_dict, eval_one, drop_togehter)

data = pd.DataFrame()
col = dict.keys()

for t, df in dict.items():
    data = pd.concat([data, df], axis=1)
data.columns = col
data.T.to_csv("{}_평균정보.csv".format(model_name))
data.T.mean().to_csv("{}_평균정보의평균.csv".format(model_name))
data.T

,BW금액/총자본,CB금액/총자본,유상증자금액/총자본,최대주주변경을수반하는주식담보제공금액/총자본,최대주주변경을수반하는주식양수도금액/총자본,타법인주식및출자증권취득결정금액/총자본,특수관계인에대한자금대여금액/총자본,당기순이익/총자본,매출채권/총자본,매출총이익/총자본,...,상호변경카운트,업종변경카운트,조회공시요구카운트,최대주주변경카운트,결산기간_거래량증가,결산기간_주가변동률,차입금의존도,총자본,최대주주비율,최대주주비율_변동성
2012-01,55.927068,15.383557,106.205270,0.000000,33.266491,43.520008,5.225357e-07,-0.049073,0.390850,0.024642,...,0.045061,0.088388,0.466205,0.128250,1.043768,1.030201,25.122192,2.776238e+08,36.976345,0.001442
2012-04,98.128499,25.861309,58.564137,0.000000,29.790261,65.275147,1.749695e-06,-0.179461,0.584879,0.043228,...,0.050459,0.096330,0.534404,0.160550,inf,1.424649,30.372601,9.788820e+07,30.314874,0.010791
2012-07,103.046211,13.813589,38.882296,0.000000,21.833544,52.079480,2.485681e-07,-0.146189,0.392826,-0.016920,...,0.058462,0.024615,0.366154,0.118462,inf,1.334483,23.505977,2.396651e+08,35.987561,0.002403
2012-10,112.927120,13.819122,43.723488,0.000000,28.187654,75.457837,4.320723e-06,-0.167293,0.568599,0.052619,...,0.058824,0.026565,0.413662,0.129032,inf,1.505202,28.950008,7.803125e+07,31.434022,0.005247
2013-01,102.051096,30.048016,53.859441,0.000000,27.955764,87.784350,5.562140e-06,-0.308796,0.949037,0.082859,...,0.072314,0.026860,0.464876,0.167355,inf,2.101050,34.152352,1.651435e+08,30.504840,0.005690
2013-04,119.840976,32.166550,311.442719,0.000000,22.673006,90.460266,8.653522e-07,-0.214462,0.687615,0.048473,...,0.084091,0.027273,0.461364,0.200000,inf,2.194692,34.824142,1.195769e+08,29.416913,0.007891
2013-07,30.847244,41.198158,48.049394,0.000000,19.152817,35.891916,1.543497e-06,-0.077498,0.403973,0.023971,...,0.063406,0.021739,0.369565,0.159420,1.810817,2.201695,27.194380,6.183067e+08,37.131458,0.009349
2013-10,25.765730,58.493068,71.372501,0.000000,25.024126,105.812243,1.554403e-06,-0.150500,0.647875,0.039430,...,0.059426,0.020492,0.395492,0.198770,inf,1.979593,35.582256,1.884624e+08,30.455270,inf
2014-01,5.826746,32.449144,102.083679,0.000000,333.705900,56.037899,7.142201e-07,-1.327773,0.783609,-0.504373,...,0.057866,0.027125,0.332731,0.108499,1.767678,2.052471,26.972636,4.327804e+08,36.510472,inf
2014-04,17.158105,64.735267,45.364517,0.000000,30.054610,61.314180,1.667870e-06,-1.582324,1.143478,-0.544193,...,0.068966,0.028017,0.446121,0.144397,inf,1.567474,33.141067,1.427784e+08,30.754983,0.008002


In [22]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [23]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,-0.338950,0.014596,0.095082,2,2.5
A000040,0.236510,0.479381,0.414401,1,2.5
A000050,-0.067743,-0.555969,0.348926,0,2.5
A000070,-0.606419,-0.136673,0.337044,2,2.5
A000080,-0.701654,0.483895,0.309413,2,2.5
...,...,...,...,...,...
A353810,-0.713634,0.737367,-0.288143,2,2.5
A354200,0.879569,-0.223721,-0.192463,1,2.5
A357780,-0.866547,-0.245999,-0.295404,2,2.5
A363280,-0.101217,-0.895228,0.143850,0,2.5
